# Prepare dataset

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy
import random
import pickle
import math
import sklearn

training_data = []
directory = r'' # path to dataset directory here (parent of the image folders)
labels = ['label1', 'label2']
image_size = 100
perc_test = 20

def create_training_data():
    for label in labels:
        path = os.path.join(directory, label) # point correct label to correct folder
        class_number = labels.index(label)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_array = cv2.resize(img_array, (image_size, image_size))
                training_data.append([img_array, class_number])
            except Exception as e:
                print(e)

create_training_data()
random.shuffle(training_data)

In [ ]:
# Save dataset as pickle

X = []
y = []

for feature, label in training_data:
    X.append(feature)
    y.append(label)

X = numpy.array(X).reshape(-1, image_size, image_size, 1)
training_x = X[: int(len(X) - perc_test * len(X) / 100.0)]; test_x = X[int(len(X) - perc_test * len(X) / 100.0) :]
training_y = y[: int(len(X) - perc_test * len(X) / 100.0)]; test_y = y[int(len(X) - perc_test * len(X) / 100.0) :]

#train_x, test_x = sklearn.model_selection.train_test_split(X)

pickle_out = open("training_x.pickle","wb")
pickle.dump(training_x, pickle_out)
pickle_out.close()

pickle_out = open("test_x.pickle","wb")
pickle.dump(test_x, pickle_out)
pickle_out.close()

pickle_out = open("training_y.pickle","wb")
pickle.dump(training_y, pickle_out)
pickle_out.close()

pickle_out = open("test_y.pickle","wb")
pickle.dump(test_y, pickle_out)
pickle_out.close()

#print(training_x.shape, training_y.shape, test_x.shape, test_y.shape)

# Build, evaluate and save model

In [ ]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import TensorBoard
import time

model_name='test-{}'.format(time.time())

gpu_options = tensorflow.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tensorflow.Session(config=tensorflow.ConfigProto(gpu_options=gpu_options))

model = models.Sequential()
model.add(layers.Conv2D(256, (3, 3), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(labels), activation='softmax'))

#model.summary()

tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Load the pickle files:
pickle_in = open("training_x.pickle", "rb")
training_x = pickle.load(pickle_in)  # training images 
#training_x = tensorflow.keras.utils.normalize(training_x, axis=1)
training_x = training_x / 255.0

pickle_in = open("training_y.pickle", "rb")
training_y = pickle.load(pickle_in)  # training labels

pickle_in = open("test_x.pickle", "rb")
test_x = pickle.load(pickle_in)  # test images 
#test_x = tensorflow.keras.utils.normalize(test_x, axis=1)
test_x = test_x / 255.0

pickle_in = open("test_y.pickle", "rb")
test_y = pickle.load(pickle_in)  # test labels

model.fit(training_x, training_y, batch_size=64, epochs=5, validation_split=0.2, callbacks=[tensorboard])

# Should also evaluate model
test_loss, test_acc = model.evaluate(test_x, test_y)
print('Loss: {}, Accuracy: {}'.format(test_loss, test_acc))

model.save('{}.model'.format(model_name))

# Test model

In [ ]:
def prepare(path):
    img_array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_array = cv2.resize(img_array, (image_size, image_size))
    return img_array.reshape(-1, image_size, image_size, 1)


model = tensorflow.keras.models.load_model('test1.model')
path = r'' # path to test images here

predictions = model.predict([prepare(path)])
print(labels[numpy.argmax(predictions[0])])